In [4]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pymysql
import pylab as pl
import numpy as np
import pandas as pd
import os
import csv
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
import seaborn as sns
import json
from decimal import Decimal
import nltk
from nltk.corpus import stopwords
import statsmodels.api as sm
from wordcloud import WordCloud 
import math, re, string, requests, json
from itertools import product
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
plt.rcParams['figure.figsize'] = (30, 6)
from afinn import Afinn
afinn=Afinn()

C:\Users\nehah\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\nehah\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
# Connect to the database.
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             password='neha',                             
                             db='simpsons',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
 
print ("connect successful!!")

connect successful!!


In [20]:
def clean_text(normalized_text):
    nopunc = [char for char in normalized_text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    list_without_stopwords = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return ' '.join(list_without_stopwords)


In [6]:
#Sentiment calculator

# definig constants

B_INCR = 0.293
B_DECR = -0.293
N_SCALAR = -0.74

NEGATE = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

SPECIAL_CASE_IDIOMS = {"the shit": 3, "the bomb": 3, "bad ass": 1.5, "yeah right": -2,
                        "kiss of death": -1.5, "shut your mouth": -2, "hang your head": -2,
                       "you jerk": -2.5, "dee-lish": 2,"black private dick":-4, "damn right": 3,
                      "shut up": -1.5}

BOOSTER_DICT = \
{"absolutely": B_INCR, "amazingly": B_INCR, "awfully": B_INCR, "completely": B_INCR, "considerably": B_INCR,
 "decidedly": B_INCR, "deeply": B_INCR, "effing": B_INCR, "enormously": B_INCR,
 "entirely": B_INCR, "especially": B_INCR, "exceptionally": B_INCR, "extremely": B_INCR,
 "fabulously": B_INCR, "flipping": B_INCR, "flippin": B_INCR,
 "fricking": B_INCR, "frickin": B_INCR, "frigging": B_INCR, "friggin": B_INCR, "fully": B_INCR, "fucking": B_INCR,
 "greatly": B_INCR, "hella": B_INCR, "highly": B_INCR, "hugely": B_INCR, "incredibly": B_INCR,
 "intensely": B_INCR, "majorly": B_INCR, "more": B_INCR, "most": B_INCR, "particularly": B_INCR,
 "purely": B_INCR, "quite": B_INCR, "really": B_INCR, "remarkably": B_INCR,
 "so": B_INCR, "substantially": B_INCR,
 "thoroughly": B_INCR, "totally": B_INCR, "tremendously": B_INCR,
 "uber": B_INCR, "unbelievably": B_INCR, "unusually": B_INCR, "utterly": B_INCR,
 "very": B_INCR,
 "almost": B_DECR, "barely": B_DECR, "hardly": B_DECR, "just enough": B_DECR,
 "kind of": B_DECR, "kinda": B_DECR, "kindof": B_DECR, "kind-of": B_DECR,
 "less": B_DECR, "little": B_DECR, "marginally": B_DECR, "occasionally": B_DECR, "partly": B_DECR,
 "scarcely": B_DECR, "slightly": B_DECR, "somewhat": B_DECR,
 "sort of": B_DECR, "sorta": B_DECR, "sortof": B_DECR, "sort-of": B_DECR,
"unbeliev": B_INCR , "better":B_INCR ,"great": B_INCR }





def scalar_inc_dec(word, valence):
   
    scalar = 0.0
    
    if word in BOOSTER_DICT:
        scalar = BOOSTER_DICT[word]
        if valence < 0:
            scalar *= -1
   
    return scalar

def negated(input_words, include_nt=True):
    """
    Determine if input contains negation words
    """
    neg_words = []
    neg_words.extend(NEGATE)
    for word in neg_words:
        if word in input_words:
            return True
    if include_nt:
        for word in input_words:
            if "n't" in word:
                return True
    if "least" in input_words:
        i = input_words.index("least")
        if i > 0 and input_words[i-1] != "at":
            return True
    return False

def _never_check(valence, words, start_i, i):
        if start_i == 0:
            if negated([words[i-1]]):
                    valence = valence*N_SCALAR
        return valence


def _least_check(valence, words, i):
        # check for negation case using "least"
        if i > 1 and words[i-1].lower() not in lex_dict \
           and words[i-1].lower() == "least":
            if words[i-2].lower() != "at" and words[i-2].lower() != "very":
                valence = valence*N_SCALAR
        elif i > 0 and words[i-1].lower() not in lex_dict \
             and words[i-1].lower() == "least":
            valence = valence*N_SCALAR
        return valence


def _but_check(words, sentiments):
        # check for modification in sentiment due to contrastive conjunction 'but'
        if 'but' in words or 'BUT' in words:
            try:
                bi = words.index('but')
            except ValueError:
                bi = words.index('BUT')
            for sentiment in sentiments:
                si = sentiments.index(sentiment)
                if si < bi:
                    sentiments.pop(si)
                    sentiments.insert(si, sentiment*0.5)
                elif si > bi:
                    sentiments.pop(si)
                    sentiments.insert(si, sentiment*1.5)
        return sentiments
    

def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score/math.sqrt((score*score) + alpha)
    if norm_score < -1.0: 
        return -1.0
    elif norm_score > 1.0:
        return 1.0
    else:
        return norm_score

def _sift_sentiment_scores(sentiments):
        # want separate positive versus negative sentiment scores
        pos_sum = 0.0
        neg_sum = 0.0
        neu_count = 0
        for sentiment_score in sentiments:
            if sentiment_score > 0:
                pos_sum += (float(sentiment_score) +1) # compensates for neutral words that are counted as 1
            if sentiment_score < 0:
                neg_sum += (float(sentiment_score) -1) # when used with math.fabs(), compensates for neutrals
            if sentiment_score == 0:
                neu_count += 1
        return pos_sum, neg_sum, neu_count

def sentiment_valence(valence,item, i, sentiments,words):
        
        if item in lex_dict:
            #get the sentiment valence
            valence = lex_dict[item]

            for start_i in range(0,3):
                if i > start_i and words[i-(start_i+1)].lower() not in lex_dict:
                    # dampen the scalar modifier of preceding words
                    # (excluding the ones that immediately preceed the item) based
                    # on their distance from the current item.
                    #print(i)
                    s = scalar_inc_dec(words[i-(start_i+1)], valence)
                    if start_i == 1 and s != 0:
                        s = s*0.95
                    if start_i == 2 and s != 0:
                        s = s*0.9
                    valence = valence+s
                    #print(valence)
                    valence = _never_check(valence, words, start_i, i)
                    #if start_i == 2:
                        #valence = _idioms_check(valence, words, i)
                        
            valence = _least_check(valence, words, i)

        sentiments.append(valence)
        return sentiments

def score_valence(sentiments, text):
        if sentiments:
            sum_s = float(sum(sentiments))

            compound = normalize(sum_s)
            # discriminate between positive, negative and neutral sentiment scores
            pos_sum, neg_sum, neu_count = _sift_sentiment_scores(sentiments)

            total = pos_sum + math.fabs(neg_sum) + neu_count
            pos = math.fabs(pos_sum / total)
            neg = math.fabs(neg_sum / total)
            neu = math.fabs(neu_count / total)

        else:
            compound = 0.0
            pos = 0.0
            neg = 0.0
            neu = 0.0

        sentiment_dict = \
            {"neg" : round(neg, 3),
             "neu" : round(neu, 3),
             "pos" : round(pos, 3),
             "compound" : round(compound, 4)}

        return sentiment_dict


def polarity_scores(text):
    #text=clean_text(text)
    
    sentiments = []
    words=text.split()
    #words=[stemmer.stem(word) for word in words]
    for item in words:
        valence=0
        i=words.index(item)
        #print(i)
        sentiments = sentiment_valence(valence,item, i, sentiments,words)
        sentiments = _but_check(words, sentiments)
    valence_dict = score_valence(sentiments, text)

    return valence_dict['compound']

Load Data

Load the lexicon file

In [7]:
#C:\Users\nehah\Desktop\Simpsons Project\Datasets
file1 = open("C:/Users/nehah/Desktop/Simpsons Project/Datasets/Simpsons Lexicon.csv","r")
lex_dict = {}
for line in file1:
    (word, measure) = line.strip().split(',')[0:2]
    lex_dict[word] = float(measure)

In [8]:
df_script_lines=pd.read_sql("select normalized_text from script_lines",con=connection);

df_script_lines=df_script_lines[df_script_lines['normalized_text'] != '']

df_script_lines['normalized_text_stem'] =  [" ".join([stemmer.stem(word) for word in sentence.split(" ")]) for sentence in df_script_lines['normalized_text']]

df_script_lines['sentiment_score']  =  df_script_lines['normalized_text'].map(polarity_scores)

df_script_lines['normalized_text_stem'].head(10)

2                                        ooo care homer
3                              there no time to be care
4                                             were late
7                              sorri excus us pardon me
8     hey norman how it go so you got drag down here...
9                                      pardon my galosh
10    wasnt that wonder and now santa of mani land a...
11                                        oh lisa class
12    frohlich weihnachten -- that german for merri ...
13    meri kurimasu i am hotseiosha a japanes priest...
Name: normalized_text_stem, dtype: object

In [9]:
#print(lex_dict.keys())

In [10]:
def clean_words(normalized_text):
    clean=[word for word in normalized_text.split(' ') if word in lex_dict.keys()]
    return ' '.join(clean)

In [11]:
df_script_lines['normalized_text_stem1']=df_script_lines['normalized_text_stem'].map(clean_words)

In [12]:
df_script_lines=df_script_lines[df_script_lines['sentiment_score'] !=0]

In [13]:
df_script_lines=df_script_lines.sort_values(by=['sentiment_score'], ascending=False)

In [14]:
def sentiment_score_to_category(score):
    if(score>0):
        return 'positive'
    if(score<0):
        return 'negative'
    if(score==0):
        return 'neutral'

In [17]:
df_script_lines['sentiment'] =  df_script_lines['sentiment_score'].map(sentiment_score_to_category)
df_script_lines.head(5)

,normalized_text,normalized_text_stem,sentiment_score,normalized_text_stem1,sentiment
96334,hello im the us constitution and im over two h...,hello im the us constitut and im over two hund...,1.0,fine wish,positive
2461,huh a very thoughtful gift but its a surprise ...,huh a veri thought gift but it a surpris you k...,1.0,gift surpris beauti ill block,positive
34898,but i like that film biography idea a slick ho...,but i like that film biographi idea a slick ho...,1.0,like evil like,positive
35047,no homer i wont make fun of you but i will sug...,no homer i wont make fun of you but i will sug...,1.0,no fun better,positive
10433,all right theres no check just a card but dont...,all right there no check just a card but dont ...,1.0,no panic sure dear thank,positive


The above results show a clear glimpse of the original dialogue, the root words , the sentiment score and the words resulting for the sentiment.
Though the booster words, negate words, 'but' and 'never' also affect the sentiment. as of now let us continue and check the accuracy.

I will be considering the normalized_text_stem1 column as my predictor variable and score (positive or negative) as the column to be predicted.

Split data into train and test

In [18]:
text_train,text_test,sentiment_train,sentiment_test = train_test_split(df_script_lines['normalized_text_stem1'],df_script_lines['sentiment'],test_size=0.3)

Using Naive Bayes classifier

In [21]:

pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=clean_text)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
])
pipeline.fit(text_train,sentiment_train)
sentiment_predictions = pipeline.predict(text_test)


print(classification_report(sentiment_test,sentiment_predictions))

             precision    recall  f1-score   support

   negative       0.61      0.15      0.25      9076
   positive       0.63      0.94      0.75     14030

avg / total       0.62      0.63      0.55     23106



In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(sentiment_test, sentiment_predictions, normalize=True)

0.62871115727516658

Using Decision Tree Classifier

In [23]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=clean_text)),
    ('tfidf',TfidfTransformer()),
    ('classifier',DecisionTreeClassifier())
])
pipeline.fit(text_train,sentiment_train)
sentiment_predictions = pipeline.predict(text_test)


print(classification_report(sentiment_test,sentiment_predictions))

             precision    recall  f1-score   support

   negative       0.80      0.61      0.69      9076
   positive       0.78      0.90      0.84     14030

avg / total       0.79      0.78      0.78     23106



In [24]:
accuracy_score(sentiment_test, sentiment_predictions, normalize=True)

0.78486107504544278

Using Random Forest classifier 

In [25]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=clean_text)),
    ('tfidf',TfidfTransformer()),
    ('classifier',RandomForestClassifier())
])
pipeline.fit(text_train,sentiment_train)
sentiment_predictions = pipeline.predict(text_test)


print(classification_report(sentiment_test,sentiment_predictions))

             precision    recall  f1-score   support

   negative       0.82      0.61      0.70      9076
   positive       0.79      0.92      0.85     14030

avg / total       0.80      0.80      0.79     23106



In [26]:
accuracy_score(sentiment_test, sentiment_predictions, normalize=True)

0.79641651519085954

In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(sentiment_test, sentiment_predictions)

array([[ 5564,  3512],
       [ 1192, 12838]])

Support Vector Machine

In [31]:
from nltk.classify import SklearnClassifier
from sklearn.svm import SVC
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=clean_text)),
    ('tfidf',TfidfTransformer()),
    ('classifier',SVC())
])
pipeline.fit(text_train,sentiment_train)
sentiment_predictions = pipeline.predict(text_test)


print(classification_report(sentiment_test,sentiment_predictions))

             precision    recall  f1-score   support

   negative       0.70      0.40      0.51      9076
   positive       0.70      0.89      0.78     14030

avg / total       0.70      0.70      0.68     23106



In [32]:
accuracy_score(sentiment_test, sentiment_predictions, normalize=True)

0.69864970137626592

It is observed that the best results 80% score has been obtained using the Random Forest Classifier which is more or less same as that obtained from Decision Tree.
Hence, we can colclude that random forest classifier is thebest suit for our dataset as of now.
There is more scope for accurate classification of this data so as to get better results.